In [25]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [26]:
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Combine the training and testing data
x = np.concatenate((x_train, x_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

# Shuffle the data randomly
np.random.seed(42)
shuffle_idx = np.random.permutation(len(x))
x_shuffled = x[shuffle_idx]
y_shuffled = y[shuffle_idx]

# Split the data into 67% training and 33% testing
split_idx = int(0.67 * len(x_shuffled))
x_train, y_train = x_shuffled[:split_idx], y_shuffled[:split_idx]
x_test, y_test = x_shuffled[split_idx:], y_shuffled[split_idx:]

In [27]:
# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

In [28]:
# Define function to build neural network
def build_model(num_hidden_layers, num_neurons, activation_func):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    for i in range(num_hidden_layers):
        model.add(keras.layers.Dense(num_neurons, activation=activation_func))
    model.add(keras.layers.Dense(10, activation='softmax'))
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [37]:
# Train and evaluate 15 classifiers
#Adam optimizaer used by default
count=1
num_hidden_layers_list = [2, 3]
num_neurons_list = [100, 150]
activation_func_list = ['tanh', 'sigmoid', 'relu']

best_accuracy = 0
best_classifier = None
acc_list = []

for num_hidden_layers in num_hidden_layers_list:
    for num_neurons in num_neurons_list:
        for activation_func in activation_func_list:
            print('Model',count)
            model = build_model(num_hidden_layers, num_neurons, activation_func)
            model.fit(x_train, y_train, epochs=10)
            test_loss, test_acc = model.evaluate(x_test, y_test)
            acc_list.append(test_acc)
            if test_acc > best_accuracy:
                best_accuracy = test_acc
                best_classifier = (num_hidden_layers, num_neurons, activation_func)
            
            # Print model information and performance metrics
            print(f'Classifier: {num_hidden_layers} hidden layers, {num_neurons} neurons, {activation_func} activation function')
            print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')
            y_pred = np.argmax(model.predict(x_test), axis=-1)
            confusion_matrix = tf.math.confusion_matrix(y_test, y_pred)
            print(f'Confusion matrix:\n{confusion_matrix}\n')
            count=count+1

Model 1
Epoch 1/10
1466/1466 [==============================] - 1s 825us/step - loss: 0.2929 - accuracy: 0.9139
Epoch 2/10
1466/1466 [==============================] - 1s 898us/step - loss: 0.1371 - accuracy: 0.9590
Epoch 3/10
1466/1466 [==============================] - 1s 889us/step - loss: 0.0935 - accuracy: 0.9712
Epoch 4/10
1466/1466 [==============================] - 1s 821us/step - loss: 0.0695 - accuracy: 0.9779
Epoch 5/10
1466/1466 [==============================] - 1s 820us/step - loss: 0.0509 - accuracy: 0.9845
Epoch 6/10
1466/1466 [==============================] - 1s 820us/step - loss: 0.0386 - accuracy: 0.9880
Epoch 7/10
1466/1466 [==============================] - 1s 825us/step - loss: 0.0322 - accuracy: 0.9898
Epoch 8/10
1466/1466 [==============================] - 1s 826us/step - loss: 0.0237 - accuracy: 0.9927
Epoch 9/10
1466/1466 [==============================] - 1s 824us/step - loss: 0.0206 - accuracy: 0.9934
Epoch 10/10
722/722 [==============================] - 0

In [38]:
# Print best classifier
print(f'Best classifier: {best_classifier}, Test accuracy: {best_accuracy}')

Best classifier: (2, 150, 'tanh'), Test accuracy: 0.9770562648773193


In [40]:
count=1
num_hidden_layers = best_classifier[0]
num_neurons = best_classifier[1]
activation_func = best_classifier[2]

best_accuracy = 0

learning_rate_list = [0.01, 0.001, 0.0001]
batch_size_list = [32, 64, 128]

for learning_rate in learning_rate_list:
    for bs in batch_size_list:
        print('Model',count)
        model = build_model(num_hidden_layers, num_neurons, activation_func)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
        model.fit(x_train, y_train, epochs=10, batch_size=bs)
        test_loss, test_acc = model.evaluate(x_test, y_test, batch_size=bs)
        acc_list.append(test_acc)
        if test_acc > best_accuracy:
            best_accuracy = test_acc
            best_classifier = (num_hidden_layers, num_neurons, activation_func, learning_rate, bs)
        print(f'Classifier: {num_hidden_layers} hidden layers, {num_neurons} neurons, {activation_func} activation function, {lr} learning rate, {bs} batch size')
        print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')
        y_pred = np.argmax(model.predict(x_test), axis=-1)
        confusion_matrix = tf.math.confusion_matrix(y_test, y_pred)
        print(f'Confusion matrix:\n{confusion_matrix}\n')
        count += 1

Model 1
Epoch 1/10
1466/1466 [==============================] - 2s 1ms/step - loss: 0.3466 - accuracy: 0.8972
Epoch 2/10
1466/1466 [==============================] - 2s 1ms/step - loss: 0.2973 - accuracy: 0.9130
Epoch 3/10
1466/1466 [==============================] - 2s 1ms/step - loss: 0.2907 - accuracy: 0.9172
Epoch 4/10
1466/1466 [==============================] - 2s 1ms/step - loss: 0.2821 - accuracy: 0.9192
Epoch 5/10
1466/1466 [==============================] - 2s 1ms/step - loss: 0.2996 - accuracy: 0.9144
Epoch 6/10
1466/1466 [==============================] - 2s 1ms/step - loss: 0.2991 - accuracy: 0.9169
Epoch 7/10
1466/1466 [==============================] - 2s 1ms/step - loss: 0.2837 - accuracy: 0.9203
Epoch 8/10
1466/1466 [==============================] - 2s 1ms/step - loss: 0.2782 - accuracy: 0.9216
Epoch 9/10
1466/1466 [==============================] - 2s 1ms/step - loss: 0.2792 - accuracy: 0.9220
Epoch 10/10
722/722 [==============================] - 0s 552us/step - los

In [41]:
# Print best classifier
print(f'Best classifier: {best_classifier}, Test accuracy: {best_accuracy}')

Best classifier: (2, 150, 'tanh', 0.001, 128), Test accuracy: 0.9752380847930908


In [50]:
from scipy.stats import ttest_ind

best_acc = max(acc_list)

for i, acc in enumerate(acc_list):
    if ttest_ind(acc_list, best_acc)[1] > 0.05:
        print(f"Classifier {i+1} is not statistically significant from the best classifier")
    else:
        print(f"Classifier {i+1} is statistically significant from the best classifier")
        
#best classifier is superior to all other classifiers generated.

Classifier 1 is statistically significant from the best classifier
Classifier 2 is statistically significant from the best classifier
Classifier 3 is statistically significant from the best classifier
Classifier 4 is statistically significant from the best classifier
Classifier 5 is statistically significant from the best classifier
Classifier 6 is statistically significant from the best classifier
Classifier 7 is statistically significant from the best classifier
Classifier 8 is statistically significant from the best classifier
Classifier 9 is statistically significant from the best classifier
Classifier 10 is statistically significant from the best classifier
Classifier 11 is statistically significant from the best classifier
Classifier 12 is statistically significant from the best classifier
Classifier 13 is statistically significant from the best classifier
Classifier 14 is statistically significant from the best classifier
Classifier 15 is statistically significant from the best 

/tmp/ipykernel_143853/2637218117.py:6: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  if ttest_ind(acc_list, best_acc)[1] > 0.05:
